In [1]:
import sys
sys.path.append("/data/Beatriz/Doctorado GR/ADL_platform//S-ADL")


In [2]:
from SADL.time_series.time_series_datasets_uci import global_load as load_time_series 
from sklearn.model_selection import train_test_split
X,y = load_time_series('ai4i_2020_predictive_maintenance_dataset')   #name dataset in static datasets uci repo
labels = y["Machine failure"]
X = X.drop('Type',axis=1)  # remove Type or apply one hot encoding

2025-07-17 20:39:09.811992: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-17 20:39:09.823887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752777549.841259  150795 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752777549.845073  150795 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-17 20:39:09.860682: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Metadata: {'uci_id': 601, 'name': 'AI4I 2020 Predictive Maintenance Dataset', 'repository_url': 'https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/601/data.csv', 'abstract': 'The AI4I 2020 Predictive Maintenance Dataset is a synthetic dataset that reflects real predictive maintenance data encountered in industry.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression', 'Causal-Discovery'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 10000, 'num_features': 6, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], 'index_col': ['UID', 'Product ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2020, 'last_updated': 'Wed Feb 14 2024', 'dataset_doi': '10.24432/C5HS5C', 'creators': [], 'intro_paper': {'ID': 386, 'type': 'NATIVE', 'title': 'Explainable Artificial 

In [3]:
from SADL.time_series.preprocessing.preprocessing_ts import StandardScalerPreprocessing 
scaler = StandardScalerPreprocessing()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )


(8000, 5) (2000, 5) (8000,) (2000,)


In [10]:
from SADL.time_series.algorithms import transformers
from SADL.time_series.time_series_utils import TimeSeriesProcessor
import numpy as np
import torch

batch_size = 16
seq_len = 24
input_dim = 5  # tanto encoder como decoder
d_model = 64
train_epochs = 10

# Simulamos series de entrada
X = torch.randn(batch_size, seq_len, input_dim)

# Instanciamos el modelo Transformer
kwargs ={
    "algorithm_": "Transformer",
    "label_parser": None,
    "size_enc_in":input_dim,
    "size_dec_in": input_dim,
    "ulayers_feedfwd": 128,
    "seq_len":seq_len,
    "d_qk":64,
    "d_v":64,
    "d_model":d_model,
    "n_layers": 2,
    "n_heads": 8,
   "dropout_rate": 0.1,
   "attns_outs":False, 
    "train_epochs": train_epochs,
    "batch_size": batch_size,
    "lr": 1e-3
    }
    

model = transformers.TransformersAnomalyDetection(**kwargs)

Train Params: {'label_parser': None, 'train_epochs': 5, 'batch_size': 16, 'lr': 0.001} 
Model Params: {'algorithm_': 'Transformer', 'size_enc_in': 5, 'size_dec_in': 5, 'ulayers_feedfwd': 128, 'seq_len': 24, 'd_qk': 64, 'd_v': 64, 'd_model': 64, 'n_layers': 2, 'n_heads': 8, 'dropout_rate': 0.1, 'attns_outs': False}


In [12]:
processor = TimeSeriesProcessor(window_size= seq_len, step_size=1, future_prediction=False)
X_train_windows, y_train_windows, X_test_windows, y_test_windows = processor.process_train_test(X_train, y_train, X_test, y_test)
print("X_train shape:", X_train_windows.shape)
print("y_train shape:", y_train_windows.shape)
print("X_test shape:", X_test_windows.shape)
print("y_test shape:", y_test_windows.shape)

X_train shape: (7977, 24, 5)
y_train shape: (7977, 24)
X_test shape: (1977, 24, 5)
y_test shape: (1977, 24)


In [13]:

model.fit(X_train_windows)


/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/data/Beatriz/Doctorado GR/ADL_platform/S-ADL/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([9, 24, 5])) that is different to the input size (torch.Size([24, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/5, Loss: 1.004393
Epoch 2/5, Loss: 0.980808
Epoch 3/5, Loss: 0.979639
Epoch 4/5, Loss: 0.979225
Epoch 5/5, Loss: 0.973495


In [ ]:
scores = model.decision_function(X_train_windows)
labels = model.predict(X_train_windows)

print("Anomaly scores:", scores.shape)
print("Predicted labels:", labels.shape)


In [ ]:
model.evaluate(X_test_windows,y_test_windows)